In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
import sys
import pandas as pd

sys.path.append('../../../param_search')
import param_search as ps

ps.set_verbose(False)
ps.set_backend('slurm')

In [3]:
csv_path = '2025-11-02_selected.csv'
sel = pd.read_csv(csv_path)
sel

,subject,metadata.category.set,metadata.category.len,metadata.category.set_f,metadata.category.len_f,metadata.category.valid,metadata.category.reasons,metadata.unit.value,metadata.unit.valid,metadata.unit.reasons,...,artifacts.valid,artifacts.reasons,valid,reasons,artifacts.scene.exc,paths.source_mask.exc,source_path,job_name,job_id,dims_error
0,wss.100f39dce7690f59efb94709f30ce0d2,"{'Recliner', 'Chair'}",2,"{'Recliner', 'Chair'}",2,True,(),0.012947,True,(),...,True,(),True,(),NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,0103f8f8d5c25bf7,35732758,7.294375e-08
1,wss.101354f9d8dede686f7b08d9de913afe,"{'Speaker', '_Attributes'}",2,{'Speaker'},1,True,(),0.013620,True,(),...,True,(),True,(),NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,0103f8f8d5c25bf7,35732758,1.319075e-08
2,wss.1029fd5d71a06ea32ea98d69e91ba870,{'ChestOfDrawers'},1,{'ChestOfDrawers'},1,True,(),0.019631,True,(),...,True,(),True,(),NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,0103f8f8d5c25bf7,35732758,8.319422e-08
3,wss.102a6b7809f4e51813842bc8ef6fe18,"{'Desktop', 'Computer'}",2,"{'Desktop', 'Computer'}",2,True,(),0.025252,True,(),...,True,(),True,(),NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,0103f8f8d5c25bf7,35732758,1.077898e-07
4,wss.107637b6bdf8129d4904d89e9169817b,{'Couch'},1,{'Couch'},1,True,(),0.022625,True,(),...,True,(),True,(),NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,0103f8f8d5c25bf7,35732758,7.520307e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,wss.d81f943065391492a7e760c0b399301,"{'FoodItem', 'Bowl', '_StanfordSceneDBModels',...",4,"{'FoodItem', 'Bowl', 'FruitBowl'}",3,True,(),0.001381,True,(),...,True,(),False,"('missing dims',)",NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,5c023e2f832d4dcf,35732849,NaN
1112,wss.d8d4004791c4f61b80fa98b5eeb7036c,"{'_StanfordSceneDBModels', 'Book'}",2,{'Book'},1,True,(),0.000002,True,(),...,True,(),False,"('missing dims',)",NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,5c023e2f832d4dcf,35732849,NaN
1113,wss.e8add7acb28e2505fd40810f48721e8b,"{'Bed', 'DoubleBed'}",2,"{'Bed', 'DoubleBed'}",2,True,(),0.009245,True,(),...,True,(),False,"('missing dims',)",NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,5c023e2f832d4dcf,35732849,NaN
1114,wss.f6420a3a3addd027f67be63aca8832dd,"{'Recliner', 'Chair'}",2,"{'Recliner', 'Chair'}",2,True,(),0.027262,True,(),...,True,(),False,"('missing dims',)",NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,8786de37533664e5,35732850,NaN


In [4]:
base_dir = '2025-11-02_preprocess'

template = '''\
#!/bin/bash -l
#SBATCH --job-name={job_name}
#SBATCH --account=asc170022p
#SBATCH --partition=GPU-shared
#SBATCH --gres=gpu:1
#SBATCH -t 24:00:00
set -eo pipefail

LP=$PROJECT/lung-project
NB=$PROJECT/lung-project/notebooks/shapenet

mamba activate /ocean/projects/asc170022p/mtragoza/mambaforge/envs/warp

python $LP/preprocess.py --dataset shapenet --data_root $LP/data/ShapeNetSem --variant {variant} --config $NB/{config} --subject {subject}

'''
name_format = '{params_hash}'

grid = ps.param_grid(subject=sel.subject, variant='2025-11-02', config='2025-11-02_config.json')
len(grid)

1116

In [5]:
%autoreload
try:
    jobs = ps.setup(base_dir, template, name_format, grid, overwrite=False)
except OSError:
    jobs = ps.load(base_dir)

jobs

,job_name,job_state,n_submits,job_id,node_id,runtime,stdout,stderr,base_dir,work_dir,...,params.config,output_exists,output_fsize,output_mtime,array_idx,last_live_at,state_source,finalized,finalized_at,error_line
0,29ce194442049995,COMPLETED,1,35760489,v031,00:01:28,Module warp.fem.space.restriction.dyn.fill_ele...,Total optimization time: 9.98211s\n\nRunning s...,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,2025-11-02_config.json,False,None,None,NaN,2025-11-02T20:25:30,history,True,2025-11-02T20:29:56,None
1,14808a4dca193cc6,COMPLETED,1,35760490,v031,00:01:38,"(224, 224, 224, 3) float32\nE (6846, 1) float3...",Total optimization time: 9.29207s\n\nRunning s...,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,2025-11-02_config.json,False,None,None,NaN,2025-11-02T20:25:30,history,True,2025-11-02T20:29:56,None
2,80b6d4fad5d281e2,COMPLETED,1,35760491,v031,00:02:23,Module warp.fem.space.restriction.dyn.fill_ele...,Total optimization time: 3.41252s\n\nRunning s...,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,2025-11-02_config.json,False,None,None,NaN,2025-11-02T20:29:56,history,True,2025-11-02T20:33:26,None
3,dad19a9da26c0152,COMPLETED,1,35760492,v018,00:04:09,"E (7190, 1) float32\nmu (7190, 1) float32\nlam...",Total optimization time: 17.0064s\n\nRunning s...,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,2025-11-02_config.json,False,None,None,NaN,2025-11-02T20:33:26,history,True,2025-11-02T21:14:03,None
4,6367e780f09371ad,COMPLETED,1,35760493,w008,00:03:46,Module warp.fem.space.restriction.dyn.fill_ele...,Total optimization time: 5.37212s\n\nRunning s...,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,2025-11-02_config.json,False,None,None,NaN,2025-11-02T20:33:26,history,True,2025-11-02T21:14:03,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,4e863fd445db4f39,COMPLETED,1,35761601,v025,00:01:12,"(224, 224, 224, 3) float32\nE (6884, 1) float3...",Total optimization time: 11.9461s\n\nRunning s...,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,2025-11-02_config.json,False,None,None,NaN,2025-11-02T21:14:01,history,True,2025-11-03T12:09:17,None
1112,c1a80e8d8e56ce2a,COMPLETED,1,35761602,v025,00:04:11,Module warp.fem.integrate._identity_field__itp...,Total optimization time: 102.319s\n\nRunning s...,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,2025-11-02_config.json,False,None,None,NaN,2025-11-02T21:14:01,history,True,2025-11-03T12:09:17,None
1113,9b515b94d1331bc5,COMPLETED,1,35761603,v029,00:01:25,Rasterizing output field\nModule warp.fem.spac...,Total optimization time: 24.5643s\n\nRunning s...,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,2025-11-02_config.json,False,None,None,NaN,2025-11-02T21:14:01,history,True,2025-11-03T12:09:17,None
1114,8a767031556adad6,COMPLETED,1,35761604,v029,00:02:06,Rasterizing output field\nModule warp.fem.spac...,Total optimization time: 9.52996s\n\nRunning s...,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,2025-11-02_config.json,False,None,None,NaN,2025-11-02T21:14:01,history,True,2025-11-03T12:09:17,None


In [7]:
%autoreload
jobs = ps.recover(jobs)
jobs = ps.status(jobs)
jobs = ps.history(jobs)
#jobs = ps.collect(jobs)
jobs.groupby(['job_state']).count()

,job_name,n_submits,job_id,node_id,runtime,stdout,stderr,base_dir,work_dir,script_path,...,params.config,output_exists,output_fsize,output_mtime,array_idx,last_live_at,state_source,finalized,finalized_at,error_line
job_state,,,,,,,,,,,,,,,,,,,,,
COMPLETED,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,...,1115,1115,0,0,0,1115,1115,1115,1115,0
FAILED,1,1,1,1,1,1,1,1,1,1,...,1,1,0,0,0,1,1,1,1,0


In [9]:
jobs[jobs.job_state == 'FAILED'].iloc[0]

job_name                                           20a6e4de06069c21
job_state                                                    FAILED
n_submits                                                         3
job_id                                                     35787632
node_id                                                        v022
runtime                                                    00:00:20
stdout            |   ├── 'material_mask':     PosixPath('/ocean...
stderr              File "/ocean/projects/asc170022p/mtragoza/lu...
base_dir          /ocean/projects/asc170022p/mtragoza/lung-proje...
work_dir          /ocean/projects/asc170022p/mtragoza/lung-proje...
script_path       /ocean/projects/asc170022p/mtragoza/lung-proje...
output_path       /ocean/projects/asc170022p/mtragoza/lung-proje...
log_dir           /ocean/projects/asc170022p/mtragoza/lung-proje...
stdout_path       /ocean/projects/asc170022p/mtragoza/lung-proje...
stderr_path       /ocean/projects/asc170022p/mtr

In [8]:
print(jobs[jobs.job_state == 'FAILED'].iloc[0].stderr)

  File "/ocean/projects/asc170022p/mtragoza/lung-project/preprocess.py", line 63, in <module>
    main(sys.argv[1:])
  File "/ocean/projects/asc170022p/mtragoza/lung-project/preprocess.py", line 59, in main
    run_pipeline(ex, pipeline_cfg)
  File "/ocean/projects/asc170022p/mtragoza/lung-project/project/preprocessing/api.py", line 89, in preprocess_shapenet
    _check_output(
  File "/ocean/projects/asc170022p/mtragoza/lung-project/project/preprocessing/api.py", line 18, in _check_output
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/ocean/projects/asc170022p/mtragoza/lung-project/project/preprocessing/api.py", line 210, in create_volume_mesh_from_mask
    mesh = volume_meshing.generate_mesh_from_mask(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/ocean/projects/asc170022p/mtragoza/lung-project/project/preprocessing/volume_meshing.py", line 57, in generate_mesh_from_mask
    mesh = assign_cell_labels(mesh, mask, new_key='label', old_key='medi

In [ ]:
# mark failed jobs to resubmit
jobs.loc[(jobs.job_state == 'FAILED'), 'job_id'] = pd.NA

In [ ]:
%autoreload
jobs = ps.submit(jobs)